<a href="https://colab.research.google.com/github/xu-pu/colab-notebooks/blob/master/dynamic_system_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic System Class Definition

import the symbolic equations of motion

In [77]:
!git clone https://github.com/xu-pu/colab-notebooks

fatal: destination path 'colab-notebooks' already exists and is not an empty directory.


In [78]:
import sys
sys.path.append('colab-notebooks')

In [79]:
import dynamics.eom_double_pendulum as eom

In [80]:
import numpy as np
from sympy import lambdify, init_printing
init_printing()

# Base Class for A Dynamic System

In [81]:
from abc import ABC, abstractmethod
import inspect

In [82]:
class DynamicSystem(ABC):

  @abstractmethod
  def eom(self, t, y) -> np.ndarray:
    pass

  @property
  @abstractmethod
  def ydim(self) -> int:
    pass

# Base Class For A Plottable System

In [83]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

class PlottableSystem(DynamicSystem):

  @property
  @abstractmethod
  def bounding_box(self) -> np.ndarray:
    """return (xmin, xmax, ymin, ymax)"""
    pass

  @abstractmethod
  def y2pt(self, y) -> np.ndarray:
    pass

  def vy2vpt(self, vy):
    t, ydim = vy.shape
    assert(ydim == self.ydim)
    return np.array([self.y2pt(vy[i,:]) for i in range(t)])

  def plot(self, vy, fps: int, filename=None):
    anim = self.animate(vy,fps)
    if filename is not None:
      anim.save(filename, writer=animation.FFMpegWriter(fps=fps))
    return HTML(anim.to_jshtml())
  
  def animate(self, vy, fps: int) -> animation.FuncAnimation:
    if type(vy) == np.ndarray:
      vy = [vy,]
    T, ydim = vy[0].shape
    assert(ydim == self.ydim)

    trajs = [self.vy2vpt(traj) for traj in vy]

    # First set up the figure, the axis, and the plot element we want to animate
    fig, ax = plt.subplots()

    xmin, xmax, ymin, ymax = self.bounding_box

    ax.set_aspect('equal')
    ax.set_xlim((xmin, xmax))
    ax.set_ylim((ymin, ymax))

    lines = [ax.plot([], [], lw=1, marker="o")[0] for i in trajs]

    # initialization function: plot the background of each frame
    def init():
      for line in lines:
        line.set_data([], [])
        return lines

    # animation function. This is called sequentially
    def animate(t):
      for idx, line in enumerate(lines):
        pts = trajs[idx][t,:]
        line.set_data(pts[:,0],pts[:,1])
      return lines

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(
        fig, 
        animate, 
        init_func=init, 
        frames=T,
        interval=1000/fps, 
        blit=True
    )
    
    plt.close()

    return anim

# Double Pendulum

In [84]:
class DoublePendulum(PlottableSystem):
  
  def __init__(self, m1, m2, l1, l2):
    self.l1 = l1
    self.l2 = l2
    self.m1 = m1
    self.m2 = m2
    self.g = 9.8
    self.theta = np.array([[self.m1,self.m2,self.l1,self.l2,self.g]]).T
    self.f  = lambdify([eom.theta, eom.y], eom.eom_rhs, 'numpy')
    self.x1 = lambdify([eom.theta, eom.y], eom.x1.subs(eom.rules_eom))
    self.y1 = lambdify([eom.theta, eom.y], eom.y1.subs(eom.rules_eom))
    self.x2 = lambdify([eom.theta, eom.y], eom.x2.subs(eom.rules_eom))
    self.y2 = lambdify([eom.theta, eom.y], eom.y2.subs(eom.rules_eom))

  def eom(self, t, y):
    return self.f(self.theta, y.reshape(4,1)).reshape(4,)

  @property
  def bounding_box(self):
    padding_ratio = 1.2
    radius = self.l1 + self.l2
    return padding_ratio * radius * np.array([ -1, 1, -1, 1 ])

  @property
  def ydim(self):
    return 4

  def y2pt(self, y):
    y = y.reshape((4,1))
    p1 = np.array([
        self.x1(self.theta, y), 
        self.y1(self.theta, y)
    ])
    p2 = np.array([
        self.x2(self.theta, y), 
        self.y2(self.theta, y)
    ])
    return np.array([[0,0], p1, p2])

assert(inspect.isabstract(DoublePendulum) is False)

# Simulate

In [85]:
system = DoublePendulum(1,1,1,1)
system.theta.shape

In [86]:
from scipy.integrate import solve_ivp

fps = 15
duration = 5
frames = fps * duration
interval = 1000 / fps
length = 1

def simulate(iv):
  return solve_ivp(
    system.eom, 
    (0,duration), 
    iv,
    t_eval = np.linspace(0,duration,duration*fps)
  )

ivs = [(np.deg2rad(90),np.deg2rad(offset),0,0) for offset in np.linspace(-5,5,5)]

results = [simulate(iv) for iv in ivs]

In [87]:
vy = [rst.y.T for rst in results]
system.plot(vy, fps, 'double_pendulum_animation.mp4')